In [57]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [58]:
#Importando bibliotecas
import pandas as pd
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster

In [59]:
df_lucas  = pd.read_csv('Planilha sem título - Página1.csv')
df_lucas

,Valor,Estado
0,105136.11,BA
1,14940.00,BA
2,75650.00,ES
3,49050.00,ES
4,411876.00,MA
5,103950.00,MA
6,133800.00,MG
7,99298.84,MG
8,88501.98,MG
9,31600.00,MG


In [60]:
df_lucas['Valor'] = df_lucas['Valor'].astype(float)
df_lucas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Valor   49 non-null     float64
 1   Estado  49 non-null     object 
dtypes: float64(1), object(1)
memory usage: 916.0+ bytes


In [61]:
df_lucas_grupo = df_lucas.groupby('Estado')['Valor'].sum().reset_index()
df_lucas_grupo

,Estado,Valor
0,AC,0.00
1,AL,0.00
2,AM,0.00
3,AP,0.00
4,BA,120076.11
5,CE,0.00
6,DF,0.00
7,ES,124700.00
8,GO,0.00
9,MA,515826.00


In [62]:
coordenadas_estados = {
    "BA": [-12.9704, -38.5124],
    "ES": [-20.3155, -40.3128],
    "MA": [-2.5296, -44.3022],
    "MG": [-18.5122, -44.5550],
    "MS": [-20.4697, -54.6201],
    "PE": [-8.0476, -34.8770],
    "RJ": [-22.9068, -43.1729],
    "SP": [-23.5505, -46.6333]
}

In [63]:
df_lucas_grupo["Latitude"] = df_lucas_grupo["Estado"].map(lambda x: coordenadas_estados.get(x, [None, None])[0])
df_lucas_grupo["Longitude"] = df_lucas_grupo["Estado"].map(lambda x: coordenadas_estados.get(x, [None, None])[1])
df_lucas_grupo

,Estado,Valor,Latitude,Longitude
0,AC,0.00,NaN,NaN
1,AL,0.00,NaN,NaN
2,AM,0.00,NaN,NaN
3,AP,0.00,NaN,NaN
4,BA,120076.11,-12.9704,-38.5124
5,CE,0.00,NaN,NaN
6,DF,0.00,NaN,NaN
7,ES,124700.00,-20.3155,-40.3128
8,GO,0.00,NaN,NaN
9,MA,515826.00,-2.5296,-44.3022


In [64]:
mapa = folium.Map(location=[-14.2350, -51.9253], zoom_start=4)
heat_data = df_lucas_grupo[["Latitude", "Longitude", "Valor"]].values.tolist()
HeatMap(heat_data).add_to(mapa)
mapa.save("mapa_calor.html")

ValueError: Location values cannot contain NaNs.

In [43]:
mapa

In [66]:
import plotly.express as px
import pandas as pd


# Link para o arquivo GeoJSON com os estados brasileiros
geojson_url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson"

# Criar o mapa com Plotly
fig = px.choropleth(
    df_lucas_grupo,
    geojson=geojson_url,
    locations="Estado",  # Coluna com as siglas dos estados
    featureidkey="properties.sigla",  # Chave no GeoJSON que contém as siglas
    color="Valor",  # Coluna que define as cores
    color_continuous_scale="YlOrRd",  # Escala de cores (amarelo-laranja-vermelho)
    title="Mapa de Calor por Estado",
)

# Ajustar o mapa
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0})

# Exibir o mapa
fig.show()

# Salvar o mapa como arquivo HTML
fig.write_html("mapa_calor.html")

